In [1]:
#Data saved in csv/excel
import glob, os
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [2]:
#Set two decimal points for display
pd.options.display.float_format = "{:.2f}".format

# 1. Load the data

In [67]:
data=pd.read_csv('C:/Users/tsb7592/Downloads/Basic_Desc_output.csv')

In [68]:
data['PIN']=data['PIN'].str.upper()

In [69]:
data.head()

,PIN,DeviceID,Assessment Name,Inst,ItemID,Response,Score,Position,ResponseTime,DateCreated,InstStarted,InstEnded,Age_month,Age_group
0,TX009C,E2909A06-4CF6-4F3A-A839-A30AE05F2B5B,Assessment 2,Mullen Receptive,Mullen_Rec_Title,1,NaN,1,NaN,2021-12-05 08:30:42,2021-12-05 08:30:41,2021-12-05 08:33:49,18.00,"(15, 18]"
1,TX009C,E2909A06-4CF6-4F3A-A839-A30AE05F2B5B,Assessment 2,Mullen Receptive,Mullen_Rec_Instructions,1,NaN,2,NaN,2021-12-05 08:30:43,2021-12-05 08:30:41,2021-12-05 08:33:49,18.00,"(15, 18]"
2,TX009C,E2909A06-4CF6-4F3A-A839-A30AE05F2B5B,Assessment 2,Mullen Receptive,RL13,2,0.00,3,NaN,2021-12-05 08:32:53,2021-12-05 08:30:41,2021-12-05 08:33:49,18.00,"(15, 18]"
3,TX009C,E2909A06-4CF6-4F3A-A839-A30AE05F2B5B,Assessment 2,Mullen Receptive,RL10,1,1.00,4,NaN,2021-12-05 08:32:59,2021-12-05 08:30:41,2021-12-05 08:33:49,18.00,"(15, 18]"
4,TX009C,E2909A06-4CF6-4F3A-A839-A30AE05F2B5B,Assessment 2,Mullen Receptive,RL15,2,0.00,5,NaN,2021-12-05 08:33:18,2021-12-05 08:30:41,2021-12-05 08:33:49,18.00,"(15, 18]"


# 2. Filter the inst to be IBQ

In [70]:
df= data[data.Inst=='Infant Behavior Questionnaire - Revised - Very Short Form']

In [71]:
df.PIN=df.PIN.str[:5]

In [72]:
df.Response =pd.to_numeric(df.Response)
df.Score =pd.to_numeric(df.Score)

In [73]:
df.describe()

,Response,Score,Position,ResponseTime,Age_month
count,1521.00,1393.00,1521.00,1443.00,312.00
mean,4.62,4.74,20.00,13.89,327.62
std,2.09,1.89,11.26,51.23,48.39
min,1.00,1.00,1.00,0.51,257.00
25%,3.00,3.00,10.00,4.47,281.25
50%,5.00,5.00,20.00,6.43,333.00
75%,6.00,6.00,30.00,10.92,377.00
max,8.00,7.00,39.00,1071.29,387.00


# 3. Rescore all items to ensure appropriate response coding

In [74]:
import numpy as np
#The score should equal the response for all items EXCEPT SelfReg_ibqrvsh11, for which the score should be 8-response 
df['Score'] = np.where(df.ItemID=='SelfReg_ibqrvsh11',8-df.Response, df.Response)

# 4. Calculate the three composite scores

In [75]:
a= ['SelfReg_ibqrvsh1','SelfReg_ibqrvsh2',
    'SelfReg_ibqrvsh7','SelfReg_ibqrvsh8', 
    'SelfReg_ibqrvsh13','SelfReg_ibqrvsh14', 'SelfReg_ibqrvsh15',
    'SelfReg_ibqrvsh20', 'SelfReg_ibqrvsh21',
    'SelfReg_ibqrvsh26', 'SelfReg_ibqrvsh27' ,
    'SelfReg_ibqrvsh36', 'SelfReg_ibqrvsh37']

b= ['SelfReg_ibqrvsh3', 'SelfReg_ibqrvsh4',
    'SelfReg_ibqrvsh9', 'SelfReg_ibqrvsh10',
    'SelfReg_ibqrvsh16', 'SelfReg_ibqrvsh17',  
    'SelfReg_ibqrvsh22', 'SelfReg_ibqrvsh23',  
    'SelfReg_ibqrvsh28', 'SelfReg_ibqrvsh29', 
    'SelfReg_ibqrvsh32', 'SelfReg_ibqrvsh33']


c= [ 'SelfReg_ibqrvsh5', 'SelfReg_ibqrvsh6',
    'SelfReg_ibqrvsh11', 'SelfReg_ibqrvsh12', 
    'SelfReg_ibqrvsh18', 'SelfReg_ibqrvsh19',
    'SelfReg_ibqrvsh24', 'SelfReg_ibqrvsh25',
    'SelfReg_ibqrvsh30', 'SelfReg_ibqrvsh31',
      'SelfReg_ibqrvsh34', 'SelfReg_ibqrvsh35']

In [76]:
SelfReg_SurgIBQR= df[df.ItemID.isin(a)].groupby('PIN')['Score'].mean()
SelfReg_NegAffIBQR= df[df.ItemID.isin(b)].groupby('PIN')['Score'].mean()
SelfReg_EffContIBQR= df[df.ItemID.isin(c)].groupby('PIN')['Score'].mean()


In [77]:
comp =[SelfReg_SurgIBQR,SelfReg_NegAffIBQR,SelfReg_EffContIBQR]

In [78]:
cdf = pd.concat(comp,axis=1)
cdf.columns = ['SelfReg_SurgIBQR','SelfReg_NegAffIBQR','SelfReg_EffContIBQR']

In [79]:
cdf.corr()

,SelfReg_SurgIBQR,SelfReg_NegAffIBQR,SelfReg_EffContIBQR
SelfReg_SurgIBQR,1.00,0.40,0.41
SelfReg_NegAffIBQR,0.40,1.00,0.04
SelfReg_EffContIBQR,0.41,0.04,1.00


# 5. Calculate item-rest correlations for every item – e.g., what is the Pearson correlation for SelfReg_ibqrvsh1 with the average score for the 12 other items on SelfReg_SurgIQBR. There should be 37 item-rest correlations. 

In [80]:
items=['SelfReg_ibqrvsh1', 'SelfReg_ibqrvsh2', 'SelfReg_ibqrvsh3', 'SelfReg_ibqrvsh4',
       'SelfReg_ibqrvsh5', 'SelfReg_ibqrvsh6', 'SelfReg_ibqrvsh7',
       'SelfReg_ibqrvsh8', 'SelfReg_ibqrvsh9', 'SelfReg_ibqrvsh10',
       'SelfReg_ibqrvsh11', 'SelfReg_ibqrvsh12', 'SelfReg_ibqrvsh13',
       'SelfReg_ibqrvsh14', 'SelfReg_ibqrvsh15', 'SelfReg_ibqrvsh16',
       'SelfReg_ibqrvsh17', 'SelfReg_ibqrvsh18', 'SelfReg_ibqrvsh19',
       'SelfReg_ibqrvsh20', 'SelfReg_ibqrvsh21', 'SelfReg_ibqrvsh22',
       'SelfReg_ibqrvsh23', 'SelfReg_ibqrvsh24', 'SelfReg_ibqrvsh25',
       'SelfReg_ibqrvsh26', 'SelfReg_ibqrvsh27', 'SelfReg_ibqrvsh28',
       'SelfReg_ibqrvsh29', 'SelfReg_ibqrvsh30', 'SelfReg_ibqrvsh31',
       'SelfReg_ibqrvsh32', 'SelfReg_ibqrvsh33', 'SelfReg_ibqrvsh34',
       'SelfReg_ibqrvsh35', 'SelfReg_ibqrvsh36', 'SelfReg_ibqrvsh37']

In [81]:
from scipy.stats import pearsonr

def item_rest(itemlist):

    for i in range(len(itemlist)):
        rest = itemlist.copy()
        rest.pop(i)
        list1 = df[df.ItemID==itemlist[i]].groupby('PIN')['Score'].mean()
        list2  = df[df.ItemID.isin(rest)].groupby('PIN')['Score'].mean()
    
    # Apply the pearsonr()
        corr, _ = pearsonr(list1, list2)
        print(itemlist[i], ': Pearsons correlation is %.3f ' % corr)

In [82]:
item_rest(a)

SelfReg_ibqrvsh1 : Pearsons correlation is 0.237 
SelfReg_ibqrvsh2 : Pearsons correlation is 0.180 
SelfReg_ibqrvsh7 : Pearsons correlation is 0.446 
SelfReg_ibqrvsh8 : Pearsons correlation is 0.495 
SelfReg_ibqrvsh13 : Pearsons correlation is 0.196 
SelfReg_ibqrvsh14 : Pearsons correlation is 0.285 
SelfReg_ibqrvsh15 : Pearsons correlation is 0.155 
SelfReg_ibqrvsh20 : Pearsons correlation is 0.227 
SelfReg_ibqrvsh21 : Pearsons correlation is 0.242 
SelfReg_ibqrvsh26 : Pearsons correlation is 0.376 
SelfReg_ibqrvsh27 : Pearsons correlation is 0.360 
SelfReg_ibqrvsh36 : Pearsons correlation is 0.348 
SelfReg_ibqrvsh37 : Pearsons correlation is 0.197 


In [83]:
item_rest(b)

SelfReg_ibqrvsh3 : Pearsons correlation is 0.474 
SelfReg_ibqrvsh4 : Pearsons correlation is 0.603 
SelfReg_ibqrvsh9 : Pearsons correlation is 0.343 
SelfReg_ibqrvsh10 : Pearsons correlation is 0.265 
SelfReg_ibqrvsh16 : Pearsons correlation is 0.467 
SelfReg_ibqrvsh17 : Pearsons correlation is 0.326 
SelfReg_ibqrvsh22 : Pearsons correlation is -0.052 
SelfReg_ibqrvsh23 : Pearsons correlation is 0.548 
SelfReg_ibqrvsh28 : Pearsons correlation is 0.373 
SelfReg_ibqrvsh29 : Pearsons correlation is 0.409 
SelfReg_ibqrvsh32 : Pearsons correlation is 0.406 
SelfReg_ibqrvsh33 : Pearsons correlation is 0.448 


In [84]:
item_rest(c)

SelfReg_ibqrvsh5 : Pearsons correlation is 0.262 
SelfReg_ibqrvsh6 : Pearsons correlation is 0.312 
SelfReg_ibqrvsh11 : Pearsons correlation is 0.179 
SelfReg_ibqrvsh12 : Pearsons correlation is 0.495 
SelfReg_ibqrvsh18 : Pearsons correlation is 0.530 
SelfReg_ibqrvsh19 : Pearsons correlation is 0.445 
SelfReg_ibqrvsh24 : Pearsons correlation is 0.411 
SelfReg_ibqrvsh25 : Pearsons correlation is 0.532 
SelfReg_ibqrvsh30 : Pearsons correlation is 0.437 
SelfReg_ibqrvsh31 : Pearsons correlation is 0.466 
SelfReg_ibqrvsh34 : Pearsons correlation is 0.510 
SelfReg_ibqrvsh35 : Pearsons correlation is 0.456 


# 6. Describe each of the 3 composite scores overall and by age group. NOTE: the Infant Behavior Questionnaire is not applicable to all age groups. 

In [85]:
cdf=cdf.reset_index()

In [86]:
spin = cdf.PIN.to_list()
cpin = []
for pin in spin:
    pin+='C'
    cpin.append(pin)

In [92]:
cdf.head()

,PIN,SelfReg_SurgIBQR,SelfReg_NegAffIBQR,SelfReg_EffContIBQR
0,MN001,4.62,4.50,6.42
1,MN003,4.23,3.17,6.42
2,MN004,4.46,3.33,5.25
3,MN005,3.92,4.83,5.83
4,MN008,4.08,4.42,4.58


In [90]:
child =data[data.PIN.isin(cpin)]

In [91]:
child.PIN=child.PIN.str[:5]

In [93]:
de = pd.merge(cdf, child[['PIN','Age_month','Age_group']], on=['PIN'], how ='left')

In [97]:
de

,PIN,SelfReg_SurgIBQR,SelfReg_NegAffIBQR,SelfReg_EffContIBQR,Age_month,Age_group
0,MN001,4.62,4.50,6.42,NaN,NaN
1,MN001,4.62,4.50,6.42,NaN,NaN
2,MN001,4.62,4.50,6.42,NaN,NaN
3,MN001,4.62,4.50,6.42,NaN,NaN
4,MN001,4.62,4.50,6.42,NaN,NaN
...,...,...,...,...,...,...
8843,WI032,4.62,4.75,4.67,8.00,"(6, 8]"
8844,WI032,4.62,4.75,4.67,8.00,"(6, 8]"
8845,WI032,4.62,4.75,4.67,8.00,"(6, 8]"
8846,WI032,4.62,4.75,4.67,8.00,"(6, 8]"


In [95]:
de.head()

,PIN,SelfReg_SurgIBQR,SelfReg_NegAffIBQR,SelfReg_EffContIBQR,Age_month,Age_group
0,MN001,4.62,4.50,6.42,NaN,NaN
1,MN001,4.62,4.50,6.42,NaN,NaN
2,MN001,4.62,4.50,6.42,NaN,NaN
3,MN001,4.62,4.50,6.42,NaN,NaN
4,MN001,4.62,4.50,6.42,NaN,NaN


In [96]:
de.groupby(['Age_month'])['SelfReg_SurgIBQR'].describe()

,count,mean,std,min,25%,50%,75%,max
Age_month,,,,,,,,
5.00,729.00,4.37,0.53,3.69,3.69,4.23,4.77,5.23
6.00,402.00,3.62,0.00,3.62,3.62,3.62,3.62,3.62
7.00,902.00,4.22,0.19,4.08,4.08,4.08,4.46,4.46
8.00,1114.00,4.51,0.63,3.92,3.92,4.62,4.62,5.54
10.00,1488.00,5.06,0.75,4.23,4.23,5.23,6.00,6.00
12.00,503.00,5.32,1.37,4.15,4.15,4.15,6.92,6.92
13.00,578.00,5.54,0.00,5.54,5.54,5.54,5.54,5.54
27.00,375.00,5.46,0.00,5.46,5.46,5.46,5.46,5.46
